In [1]:
import glob
import numpy as np
%config Completer.use_jedi = False

In [2]:
def scrape_eval_data(a):
    x= a.split("\n")[0].split("\t")
    #print(x)
    t1 = float(x[0].split(" ")[0].split('[')[1] ) 
    t2 = float(x[0].split(" ")[2].split(']')[0] )
    dur = float(t2-t1)
    vad= x[3].split(',')
    #print(vad)
    v= float(vad[0].split('[')[1])
    a= float(vad[1])
    d= float(vad[2].split(']')[0])
    #print(v,a,d)
    nom = x[1]
    cat = x[2]
    data_tupl = np.array([nom, dur, cat, v, a, d],dtype='object')
    return data_tupl

In [3]:
all_eval_path =[]
sessions= ["1","2","3","4","5"]
for s in sessions:
    path = "/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session"+s+"/dialog/EmoEvaluation"
    #print (path)
    all_eval_path.append(glob.glob(path+"/*.txt"))

In [4]:
all_eval_data = []
for i in range (0,5):
    l = len(all_eval_path[i])
    print("Session"+str(i+1)+"...")
    print("Total scrips = ",l)
    for j in range(0,l) :
        f1= all_eval_path[i][j]
        f = open(f1,"r")
        a= f.read()
        x = a.split("\n\n")#[2].split("\t")
        segments = len(x)
        for k in range (1,segments-1):
            all_eval_data.append(scrape_eval_data(x[k]))
        
all_eval_data = np.array(all_eval_data)   

Session1...
Total scrips =  28
Session2...
Total scrips =  30
Session3...
Total scrips =  32
Session4...
Total scrips =  30
Session5...
Total scrips =  31


In [5]:
all_eval_path

[['/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro01.txt',
  '/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro02.txt',
  '/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro03.txt',
  '/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro04.txt',
  '/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro05.txt',
  '/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro06.txt',
  '/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro07.txt',
  '/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_script01_1.txt',
  '/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_script01_2.txt',
  '/idiap/resource/dat

In [6]:
count = 0
all_wav_path =[]
sessions= ["1","2","3","4","5"]
for s in sessions:
    path = "/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session"+s+"/sentences/wav/"
    #print (path)
    all_wav_path.append(glob.glob(path+"*/*.wav"))
    count+=1
print("Total Sessions scraped: ",count)

Total Sessions scraped:  5


In [7]:
print("Session1: {}; Session2: {}; Session3: {}, Session4: {}, Session5: {}".format(len(all_wav_path[0]),len(all_wav_path[1]),len(all_wav_path[2]),len(all_wav_path[3]),len(all_wav_path[4])))
print("Total Files = ", len(all_wav_path[0])+len(all_wav_path[1])+len(all_wav_path[2])+len(all_wav_path[3])+len(all_wav_path[4]))

Session1: 1819; Session2: 1811; Session3: 2136, Session4: 2103, Session5: 2170
Total Files =  10039


In [8]:
## path_label_lst contains all the .wav file path for IEMOCAP + their labels
path_label_lst =[]
for i in range (0,5):  # looping over sessions
    for j in range (0, len(all_wav_path[i])):  # looping over all the .wav files in the session
        file_path = all_wav_path[i][j]
        nom = file_path.split('/')[-1].split('.')[0]  # getting the name of the wav file from path
        index = np.where(all_eval_data[:,0]==nom)[0][0] # 
        emotion = all_eval_data[index][2]
        path_label_lst.append([file_path,emotion])

In [9]:
path_label_lst = np.array(path_label_lst)
print("Total .wav files in IEMOCAP: ",path_label_lst.shape)
print("Default labels: ",np.unique(path_label_lst[:,1]))
print("Default Counts: ",np.unique(path_label_lst[:,1], return_counts=True)[1])

Total .wav files in IEMOCAP:  (10039, 2)
Default labels:  ['ang' 'dis' 'exc' 'fea' 'fru' 'hap' 'neu' 'oth' 'sad' 'sur' 'xxx']
Default Counts:  [1103    2 1041   40 1849  595 1708    3 1084  107 2507]


In [10]:
'''''
Considering only 5 labels out of all labels in IEMOCAP
labels :

anger -->0
happy -->1
excited -->1
neutral -->2
sad --> 3 

we club happy and excited as happy
'''''
anger = []
happy = []
neutral =[]
sad = []
for i in range (0, path_label_lst.shape[0]):
    if path_label_lst[i][1]=='ang':
        anger.append([path_label_lst[i][0],0])
    if path_label_lst[i][1]=='hap':
        happy.append([path_label_lst[i][0],1])
    if path_label_lst[i][1]=='exc':
        happy.append([path_label_lst[i][0],1])
    if path_label_lst[i][1]=='neu':
        neutral.append([path_label_lst[i][0],2])
    if path_label_lst[i][1]=='sad':
        sad.append([path_label_lst[i][0],3])
    else:
        continue

In [11]:
all_emoData= anger + happy + neutral +sad
print("Considered emotion data: ", len(all_emoData))

Considered emotion data:  5531


In [12]:
import random
random.shuffle(all_emoData)
all_emoData

[['/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_M002.wav',
  1],
 ['/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro07/Ses05F_impro07_M032.wav',
  1],
 ['/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_script03_1/Ses02M_script03_1_M019.wav',
  1],
 ['/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script02_2/Ses03F_script02_2_F020.wav',
  1],
 ['/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro06/Ses03M_impro06_F012.wav',
  3],
 ['/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro04/Ses04F_impro04_M013.wav',
  2],
 ['/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script03_2/Ses05F_script03_2_M004.wav',
  0],
 ['/idiap/resource/database/IEMOCAP/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_

In [13]:
##Creating a complete list
for data_item in all_emoData:
    data = data_item[0]
    path = data
    #print(data)
    session_id = path.split('/')[6]
    speaker_id = path.split('/')[-2]
    label = data_item[1]
    with open('/idiap/temp/tpurohit/IEMOCAP-SD/5_Emo_CompleteList.txt', 'a') as f:
            f.write("%s %s %s %s\n" %(path, session_id, speaker_id, label))
f.close()

In [14]:
five_emo_data = np.array(all_emoData)
print("5 emotion data (.wav path + label)",five_emo_data.shape)
X_path= five_emo_data[:,0]
Y_label= five_emo_data[:,1]

print("\nSanity check:")
print(X_path.shape)
print(Y_label.shape)

5 emotion data (.wav path + label) (5531, 2)

Sanity check:
(5531,)
(5531,)


In [15]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf

KFold(n_splits=5, random_state=42, shuffle=True)

In [19]:
def list_creation (X,Y, fold,path,list_type):
    for X_path , Y_label in zip(X,Y):
        with open(path+'F'+str(fold)+'/'+list_type+'.list', 'a') as f:
            f.write("%s %s\n" %(X_path, Y_label))
    f.close()

s=1
path="/idiap/temp/tpurohit/IEMOCAP-SD/lists/"
for train_index, test_index in kf.split(X_path):
    X_train, X_test = X_path[train_index], X_path[test_index]
    Y_train, Y_test = Y_label[train_index], Y_label[test_index]
    
    ### splitting TrainData into training Subset and validation Subset 80:20
    
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
    print("Session {}:".format(s))
    
    
    #### creating .list files
    
    
    list_creation(X_test, Y_test,s, path,'test')
    print("Initial Train:{} ; Initail Test:{} ".format(X_train.shape, X_test.shape))
    
    
    list_creation(x_train,y_train,s, path,'train')
    list_creation(x_val, y_val,s, path,'cv')
    print("Network Train:{} ; Network Val:{}".format(x_train.shape, x_val.shape))
    
    print("For Training:")
    print(np.unique(y_train))
    print(np.unique(y_train, return_counts=True)[1])
    
    print("For Testing:")
    print(np.unique(Y_test))
    print(np.unique(Y_test, return_counts=True)[1])
    
    print("For Validation:")
    print(np.unique(y_val))
    print(np.unique(y_val, return_counts=True)[1])
    print("\n\n")
    s+=1

Session 1:
Initial Train:(4424,) ; Initail Test:(1107,) 
Network Train:(3539,) ; Network Val:(885,)
For Training:
['0' '1' '2' '3']
[ 715 1025 1096  703]
For Testing:
['0' '1' '2' '3']
[211 335 349 212]
For Validation:
['0' '1' '2' '3']
[177 276 263 169]



Session 2:
Initial Train:(4425,) ; Initail Test:(1106,) 
Network Train:(3540,) ; Network Val:(885,)
For Training:
['0' '1' '2' '3']
[ 694 1056 1090  700]
For Testing:
['0' '1' '2' '3']
[245 303 346 212]
For Validation:
['0' '1' '2' '3']
[164 277 272 172]



Session 3:
Initial Train:(4425,) ; Initail Test:(1106,) 
Network Train:(3540,) ; Network Val:(885,)
For Training:
['0' '1' '2' '3']
[ 703 1049 1078  710]
For Testing:
['0' '1' '2' '3']
[216 345 343 202]
For Validation:
['0' '1' '2' '3']
[184 242 287 172]



Session 4:
Initial Train:(4425,) ; Initail Test:(1106,) 
Network Train:(3540,) ; Network Val:(885,)
For Training:
['0' '1' '2' '3']
[ 732 1043 1090  675]
For Testing:
['0' '1' '2' '3']
[208 319 345 234]
For Validation:
['0' '1